In [ ]:
import contextlib

In [ ]:
@contextlib.contextmanager
def context():
    print('start')
    print('a')
    try:
        yield "abc"
    finally:
        print("exit")

In [ ]:
with context():
    print('haha')

In [ ]:
type(context)

In [ ]:
def add(x,y):
    yield x+y

In [ ]:
add(1,2).__name__

In [ ]:
class ContextManager:
    def __init__(self,fn):
        self.fn = fn
    def __call__(self,*args,**kwargs):
        self.gen = self.fn(*args,**kwargs)
        return self 
    def __enter__(self):
        try:
            return next(self.gen)
        except StopIteration:
            pass
    def __exit__(self,exc_type,exc,traceback):
        try:
            return next(self.gen)
        except StopIteration:
            pass
        return False

In [ ]:
@ContextManager
def context2(x,y):
    print('start')
    print('a')
    try:
        yield x+y
    finally:
        print("exit")

In [ ]:
with context2(1,2):
    print('haha')

In [ ]:
import threading

In [ ]:
def worker():
    print('work')

In [ ]:
thread = threading.Thread(target=worker)

In [ ]:
thread.start()

In [ ]:
import threading
import logging
import importlib
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')
class Task:
    def __init__(self,name):
        self.name = name
        self.lock = threading.Lock()
def worker(tasks):
    for task in tasks:
        if task.lock.acquire(False):
            logging.info(task.name)
tasks = [Task(x) for x in range(10)]
for x in range(5):
    threading.Thread(target=worker,args=(tasks,),name='worker-{}'.format(x))

In [ ]:
logging.info('aaa')

In [ ]:
import random
class Dispatcher:
    def __init__(self):
        self.data = None
        self.event = threading.Event()
        self.cond = threading.Condition()
    
    def consumer(self):
        while not self.event.is_set():
            with self.cond:
                self.cond.wait()
                logging.info(self.data)
    def producer(self):
        for _ in range(10):
            data = random.randint(0,100)
            logging.info(data)
            self.data = data
            with self.cond:
                self.cond.notify_all()
            self.event.wait(1)
        self.event.set()

In [ ]:
d = Dispatcher()

In [ ]:
c = threading.Thread(target=d.consumer,name='consumer')

In [ ]:
p = threading.Thread(target=d.producer,name='producer')

In [ ]:
c.start()

In [ ]:
p.start()

In [ ]:
import threading

In [ ]:
import logging
import importlib

In [ ]:
importlib.reload(logging)

In [ ]:
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')

In [ ]:
logging.info('haah')

In [ ]:
def worker(n):
    logging.info(n)

In [ ]:
t = threading.Thread(target=worker,args=(3,),name='worker')

In [ ]:
t.start()

In [ ]:
t.ident

In [ ]:
threading.Thread(target=worker,args=(3,)).start()

In [ ]:
import threading
import logging
import importlib
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')


In [ ]:
import random
import datetime
def worker(event: threading.Event):
    s = random.randint(1, 5)
    #time.sleep(s)
    event.wait(s)
    event.set()         #set()调用，把所以在wait()的线程都 唤醒执行。
    logging.info('sleep {}'.format(s))
def boss(event: threading.Event):
    start = datetime.datetime.now()
    event.wait()         #主线程阻塞等待直到发生set()调用
    logging.info('worker exit {}'.format(datetime.datetime.now() - start))
event = threading.Event()
def start():
    event = threading.Event()
    b = threading.Thread(target=boss, args=(event, ), name='boss')
    b.start()
    for x in range(5):
        threading.Thread(target=worker, args=(event, ), name='worker-{}'.format(x)).start()

In [ ]:
start()

In [ ]:
import time
class Task:
    def __init__(self,name):
        self.name = name
        self.lock = threading.Lock()
tasks = [Task(x) for x in range(10)]
def worker(tasks):
    for task in tasks:
        if task.lock.acquire(False):
            logging.info(task.name)
            time.sleep(1)
for x in range(5):
    threading.Thread(target=worker,args=(tasks,),name='worker-{}'.format(x)).start()

In [ ]:
data = 0
def worker(n):
    global data
    data += n
    data -= n
    
def change(v):
    for _ in range(100):
        worker(v)

    
t1 =threading.Thread(target=change,args=(10,))
t2 = threading.Thread(target=change,args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(data)

In [ ]:
lock = threading.Lock()

In [ ]:
help(lock.acquire)

In [ ]:
lock.acquire(blocking=False)

In [ ]:
lock.acquire()

In [ ]:
lock.acquire(blocking=False)

In [ ]:
lock.release()

In [ ]:
lokc.release()

In [ ]:
Exception.__mro__

In [ ]:
KeyboardInterrupt.__mro__

In [ ]:
SystemExit.__mro__

In [ ]:
GeneratorExit.__mro__

In [ ]:
%ls

In [ ]:
dic = {}
with open('./15-22.log') as f:
    for i in f.readlines():
        key = i.split('||')[4]
        dic[key] = i 

In [ ]:
def filetodic(x):
    dic = {}
    def StrtoInt(x):
        if x == '' or x == '\n':
            return 0
        new = x.strip(' %\n\t\r')
        for i in new:
            if i not in '0123456789.' or x =='':
                return new
        return float(new)
    with open(x) as f:
        for i in f.readlines():
            sp = i.split('||') 
            dic[i] = (int(sp[3].split(':')[-1].strip('%')),StrtoInt(sp[4].split(':')[-1]),StrtoInt(sp[5].split(':')[-1]),StrtoInt(sp[6].split(':')[-1]))
    return sorted(dic,key=lambda x:(dic[x][0],dic[x][1]))


In [ ]:
dic = filetodic('./15-22.log')

In [ ]:
def StrtoInt(x):
        lst = []
        new = x.strip(' %\n\t\r')
        if x == '':
            return x
        print(new)
        for i in new:
            if i not in '0123456789.' or x =='':
                return new
        lst.append(new)
        print(lst)
        return float(new)

In [ ]:
StrtoInt('')

In [ ]:
'0%'.strip(' %')

In [ ]:
float('100')

In [ ]:
for i in '12.3':
    if i not in '0123456789.':
        print('aaaaa')
    else:
        print(i)

In [ ]:
a = '西语南美*(352主-352~353,355,357~358,360~361,363合区) 64.237.54.171 saesp_0352'

In [ ]:
help(re.split)

In [ ]:
re.split(r'\*.* (\w.*)_\d.*',a)

In [ ]:
import datetime

In [ ]:
c = datetime.datetime.now()

In [ ]:
c.timestamp()

In [ ]:
c.strptime('%H:%M:%S')

In [ ]:
help(c.strptime)

In [ ]:
import time

In [ ]:
time.time()

In [ ]:
time.clock()

In [ ]:
help(time.clock)

In [ ]:
time.localtime()

In [ ]:
a = request.urlopen('http://ip.7road.net')

In [ ]:
c  =a.read()

In [ ]:
b = c.split()

In [ ]:
b[1].decode()

In [ ]:
'中国'.encode()

In [ ]:
b[1]

In [ ]:
c

In [ ]:
g = a.read().split()[0]

In [ ]:
g

In [ ]:
from urllib import request,parse
net = '113.107.161.21'
parms = {
            'ips': net,
            'props':'ip1'
        }
url = parse.urlencode(parms)
req = request.urlopen('http://yw.7road-inc.com:8081/queryAssetsByIpJson'+'?'+url)

In [ ]:
body = req.read()

In [ ]:
import json

In [ ]:
c = json.loads(body.decode())

In [ ]:
c

In [ ]:
c['data'][0]['113.107.161.21']['ip1']

In [ ]:
type(c['code'])

In [ ]:
import time

In [ ]:
time.time()

In [ ]:
time.asctime()

In [ ]:
time.strftime('%Y-%m-%d')

* SelectSelector 实现 Select
* PollSelector  实现 poll
* EpollSelector 实现 epoll
* DevpollSelector 实现devpoll
* KqueueSelector  实现kqueue

In [1]:
import selectors,socket
selector = selectors.DefaultSelector()
def read(conn,mask):
    message = conn.recv(1024).strip()
    print('recv {}'.format(message.decode()))
    conn.send(message)
def accept(sock,mask):
    conn,_ = sock.accept()
    conn.setblocking(False)
    selector.register(conn,selectors.EVENT_READ,read)
sock = socket.socket()
sock.bind(('127.0.0.1',5555))
sock.listen()
selector.register(sock,selectors.EVENT_READ,accept)

SelectorKey(fileobj=<socket.socket fd=50, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 5555)>, fd=50, events=1, data=<function accept at 0x7f24884e7bf8>)

In [2]:
while True:
    events = selector.select()
    for key,mask in events:
        key.data(key.fileobj, mask)

recv hello
recv 
recv a
recv 
recv a
recv 
recv 
recv 
recv 
recv select
recv 


KeyboardInterrupt: 